In [70]:
import os, click, h5py, re, pickle
import numpy as np
import aux

with open(aux.path_cfgs) as f:
    cfgs=f.read().splitlines()
    

moms_target=np.array([
    [-1,0,0],[0,-1,0],[0,0,-1],[0,0,0],[0,0,1],[0,1,0],[1,0,0],
]) # do NOT change this without changing the codes


path='data_merge/NST_d_pion2pt.h5'
with h5py.File(path,'w') as fw:
    
    fw.create_dataset('cfgs',data=cfgs)
    fw.create_dataset('moms',data=[[0,0,0],[0,0,1]])
    
    dat_piC=[]
    dat_pi0_conn=[]
    dat_pi0_disc=[]
    
    for cfg in cfgs:
        
        path=f'data_post/{cfg}/P.h5_jPP-a-tensor'
        with h5py.File(path) as f:
            moms=f['moms'][:]
            for i,mom in enumerate(moms):
                dic[tuple(mom)]=i
            momMap=[dic[tuple(list(mom)+[0,0,0]+list(mom)+[0,0,0])] for mom in moms_target]
            
            t=np.array([f['data'][st]['pi+_pi+'] for st in f['data'].keys()])
            t_piC1=np.mean(t,axis=0)
            t=np.array([f['data'][st]['pi-_pi-'] for st in f['data'].keys()])
            t_piC2=np.mean(t,axis=0)
            t=(t_piC1+t_piC2)/2
            t=t[:,momMap]
            t0=t[:,3]; t1=t[:,[0,1,2,4,5,6]]
            t1=np.mean(t1,axis=1)
            t=np.array([t0,t1]).T
            dat_piC.append(t)
            
            t=np.array([f['data'][st]['pi0_pi0'] for st in f['data'].keys()])
            t=np.mean(t,axis=0)
            t=t[:,momMap]
            t0=t[:,3]; t1=t[:,[0,1,2,4,5,6]]
            t1=np.mean(t1,axis=1)
            t=np.array([t0,t1]).T
            dat_pi0_conn.append(t)
            
        path=f'data_post/{cfg}/pi0f.h5_loops-a-Nstoc100'
        with h5py.File(path) as f:
            moms=f['moms'][:]
            dic={}
            for i,mom in enumerate(moms):
                dic[tuple(mom[6:9])]=i
            momMap=[dic[tuple(mom)] for mom in moms_target]
            
            # momMap_nage=[dic[tuple(-mom)] for mom in moms_target]
            
            t_pi0f=f['data']['pi0'][:,momMap]
            t_pi0i=np.conj(t_pi0f)
            
            t=np.array([np.roll(t_pi0f,-st,axis=0)*t_pi0i[st:st+1] for st in range(len(t_pi0f))])
            t=np.mean(t,axis=0)
            t0=t[:,3]; t1=t[:,[0,1,2,4,5,6]]
            t1=np.mean(t1,axis=1)
            t=np.array([t0,t1]).T
            dat_pi0_disc.append(t)
    
    fw.create_dataset('data/piC',data=dat_piC)
    fw.create_dataset('data/pi0_conn',data=dat_pi0_conn)
    fw.create_dataset('data/pi0_disc',data=dat_pi0_disc)
    